In [3]:
import os
import time
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=r'https://example.com/callback', scope='', open_browser=False))

In [4]:
df = pd.read_csv(r"..\data\1\tracks.csv")
df['id'] = df['id'].dropna().astype(str)


In [ ]:
ids = ["35iwgR4jXetI318WEWsa1Q", "021ht4sdgPcrDgSk7JTbKY"]
print(sp.audio_features(ids))

In [ ]:
def fetch_new_info(track_id):
    results = []

    for i in range(0, len(track_id), 1):
        batch = track_id[i:i + 50]
        print("batch", batch)
        tracks = sp.tracks(batch)
        print("tracks", tracks)

        features = sp.audio_features(batch)
        print('features', features)
        input()

        for t, f in zip(tracks, features):
            if t and f: 
                results.append({
                    'id': t['id'],
                    'name': t['name'],
                    'artist': t['artists'][0]['name'],
                    'popularity': t['popularity'],
                    'danceability': f['danceability'],
                    'energy': f['energy'],
                    'valence': f['valence'],
                    'tempo': f['tempo'],
                    'acousticness': f['acousticness'],
                    'liveness': f['liveness'],
                    'speechiness': f['speechiness']
                })
        time.sleep(0.3)
    return pd.DataFrame(results)

track_ids = df['id'].dropna().astype(str).unique().tolist()
track_ids = [tid.strip() for tid in track_ids if len(tid.strip()) == 22]
df_new = fetch_new_info(track_ids)
df_new.to_csv(r"..\data\spotify_2025.csv", index=False)